In [27]:
import sympy as syms
import numpy as np
import math

from sympy import symbols
from sympy import Matrix
from sympy import latex
from sympy import solve
from sympy.matrices import eye, zeros, ones, Identity # Special matrices (eye creates explicit matrix, Identity creates matrix expression)

## Question 1
3-Node Truss Element, with shape function matrix given.
To find:
* Consistent Fully occupied mass matrix
* Lumped Mass Matrix

In [9]:
x,l,row,A = symbols('x L rho A')
N = Matrix([[1 - 3*(x/l) + 2*(x**2/l**2),4*(x/l) - 4*(x**2/l**2),-(x/l) + 2*(x**2/l**2)]])
N # Shape Function Matrix

Matrix([[1 - 3*x/L + 2*x**2/L**2, 4*x/L - 4*x**2/L**2, -x/L + 2*x**2/L**2]])

In [13]:
# Consistent Mass Matrix
M = row*A*syms.integrate(N.T*N,(x,0,l))
M

Matrix([
[2*A*L*rho/15,   A*L*rho/15,  -A*L*rho/30],
[  A*L*rho/15, 8*A*L*rho/15,   A*L*rho/15],
[ -A*L*rho/30,   A*L*rho/15, 2*A*L*rho/15]])

### Since no explicit BCs are given, diagnolisation of the consistent mass matrix is done directly

In [15]:
# Lumped Mass Matrix
MLumped = Matrix([[M[0,0] + M[0,1] + M[0,2],0,0],[0,M[1,0] + M[1,1] + M[1,2],0],[0,0,M[2,0] + M[2,1] + M[2,2]]])
MLumped

Matrix([
[A*L*rho/6,           0,         0],
[        0, 2*A*L*rho/3,         0],
[        0,           0, A*L*rho/6]])

## Question 2
![sys](Images/1.jpg)

**To Find**: The eigenfrequency of the system

**Given Stiffness and Lumped Mass Matrix for a beam element of length $l$**

In [21]:
def stiffness(k):
    E,I,l = symbols('E I l')
    # Length of Element: k*l
    l = k*l
    return (E*I/(l))*Matrix([[12/l**2,6/l,-12/l**2,6/l],[6/l,4,-6/l,2],[-12/l**2,-6/l,12/l**2,-6/l],[6/l,2,-6/l,4]])

def lumpedMass(k):
    rho,A,l = symbols('rho A l')
    return (rho*A*k*l/2)*Matrix([[1,0,0,0],[0,0,0,0],[0,0,1,0],[0,0,0,0]])

In [23]:
# Element 1
K1 = stiffness(2)
M1 = lumpedMass(2)
K1

Matrix([
[ 3*E*I/(2*l**3),  3*E*I/(2*l**2), -3*E*I/(2*l**3),  3*E*I/(2*l**2)],
[ 3*E*I/(2*l**2),         2*E*I/l, -3*E*I/(2*l**2),           E*I/l],
[-3*E*I/(2*l**3), -3*E*I/(2*l**2),  3*E*I/(2*l**3), -3*E*I/(2*l**2)],
[ 3*E*I/(2*l**2),           E*I/l, -3*E*I/(2*l**2),         2*E*I/l]])

In [26]:
# Element 2
K2 = stiffness(1)
M2 = lumpedMass(1)
K2

Matrix([
[ 12*E*I/l**3,  6*E*I/l**2, -12*E*I/l**3,  6*E*I/l**2],
[  6*E*I/l**2,     4*E*I/l,  -6*E*I/l**2,     2*E*I/l],
[-12*E*I/l**3, -6*E*I/l**2,  12*E*I/l**3, -6*E*I/l**2],
[  6*E*I/l**2,     2*E*I/l,  -6*E*I/l**2,     4*E*I/l]])

In [30]:
# Assembled Stiffness Matrix
K = Matrix([[K1[0:2,:],zeros(2,2)],[K1[2:,0:2],K1[2:,2:] + K2[0:2,0:2],K2[0:2,2:]],[zeros(2,2),K2[2:,:]]])
K

Matrix([
[ 3*E*I/(2*l**3),  3*E*I/(2*l**2), -3*E*I/(2*l**3), 3*E*I/(2*l**2),            0,           0],
[ 3*E*I/(2*l**2),         2*E*I/l, -3*E*I/(2*l**2),          E*I/l,            0,           0],
[-3*E*I/(2*l**3), -3*E*I/(2*l**2), 27*E*I/(2*l**3), 9*E*I/(2*l**2), -12*E*I/l**3,  6*E*I/l**2],
[ 3*E*I/(2*l**2),           E*I/l,  9*E*I/(2*l**2),        6*E*I/l,  -6*E*I/l**2,     2*E*I/l],
[              0,               0,    -12*E*I/l**3,    -6*E*I/l**2,  12*E*I/l**3, -6*E*I/l**2],
[              0,               0,      6*E*I/l**2,        2*E*I/l,  -6*E*I/l**2,     4*E*I/l]])

In [32]:
# Assembled Mass Matrix
m = symbols('M')
M = Matrix([[M1[0:2,:],zeros(2,2)],[M1[2:,0:2],M1[2:,2:] + M2[0:2,0:2] + m*Matrix([[1,0],[0,0]]),M2[0:2,2:]],[zeros(2,2),M2[2:,:]]])
M

Matrix([
[A*l*rho, 0,               0, 0,         0, 0],
[      0, 0,               0, 0,         0, 0],
[      0, 0, 3*A*l*rho/2 + M, 0,         0, 0],
[      0, 0,               0, 0,         0, 0],
[      0, 0,               0, 0, A*l*rho/2, 0],
[      0, 0,               0, 0,         0, 0]])

In [34]:
# Reduced System (Node 1 and Node 3 are fixed)
Kred = K[2:4,2:4]
Mred = M[2:4,2:4]
Mred

Matrix([
[3*A*l*rho/2 + M, 0],
[              0, 0]])

In [37]:
# Eigenvalue Problem
lbda = symbols('lambda') # Eigenvalues of the system
evp = Kred - lbda*Mred
evp.det()

(-36*A*E*I*l**4*lambda*rho + 243*E**2*I**2 - 24*E*I*M*l**3*lambda)/(4*l**4)